This is a intuitive explaination of the mathematics behind Representation Learning with Contrastive Predictive Coding (CPC) https://arxiv.org/pdf/1807.03748.pdf by using CPC to learn representations of sound files for the purpose of speech recognition.

The title Contrastive Predictive Coding is itself a good summary of the 3 importance concepts of the paper.

*Contrastive*: The Objective Function incorporates a version of Noise Contrastive Estimation Whereby the acheive a better, lower loss value, the model must generate a vector that is both similar to the target vector and simultaneously dissimilar to some number of negative samples. In the example here, the batch itself is used as the positive and negative samples. 

*Predictive*: The model learns high level representations of each timestep and of sequences by predicting several timesteps into the future 

*Coding*: The model learns to predict the future not at the level of the low level high dimensional inputs but at the level of high level latent space that the inputs are encoded into.  

The CPC paper combines these concepts to learn meaningful representations of sequences, images and reinforcement learning states. 

When I read this paper and tried to explain it, I felt that one particular idea was the bottelneck in understanding the paper well enough to connect the theory with the code. That idea was the relationship between the density ratio and the linear transformation of z_t+k into a scalar and taking the exponent of that scalar.

This is what i mean, the paper states that instead of modeling a generative model such as p(x|c), it models a density ratio as a log-bilinear model.

$$\frac{P(x_{t+k}|c_t)}{P(x_{t+k})} \approx e^{z^T_{t+k} W_k c_t)} $$

Where z is an encoding of x, much like texts are an encoding of speech. Here z_t+k means the z's in the future relative to a timepoint t. c_t is a summary of series of z's from z_0 to z_t, and W is a matrix transformation to make z and c the same size so that they can be compared. 

Here is an visual explaination of the density ratio:

consider this image, lets call this image X:

<img src="https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/04/Example-of-a-GAN-Generated-MNIST-Handwritten-Digit-for-a-Vector-of-Zeros-300x225.png">

It looks like either an 8 or a 9. Its kind strange looking, and you probably have seen something like it once or twice in your early years in kindergarten but in your lifespan you typically see better written numbers. If you asked a person to draw a number, any number, what is the probability they draw this image? 

The probability of X, P(X) is therefore lower than a clearly written 8 or 9. If P(8) or P(9) is around 0.1, then P(X) is more like 0.05

The variable "c" is an idea of the number that the pixels in X represent, it is a high level concept, it doesnt bother itself with arranging the pixels in X just right and low level tasks of getting all the correlations between pixels just right to draw out a number. Imagine c = "9", c = "8" and c = "7". Clearly X is more likely 8 or 9 than it is 7. so P(X| c = 8) > P (X| c = 7).

So is c = 8 a good "idea" for what X is ? Since X is kinda strange P(X| c = 8) is still pretty low, say P(X| c = 8) = 0.01 , thats pretty terrible, and not really fair to c = 8, since the idea of "8" is not only an actualy number, as opposed to "A", it is also what a kindergartener might draw when you ask them to draw 8. 

So lets normalize this by dividing by P(x), this ratio gives us a better idea of how good c is at representing the higher level concept of X. P(X| c = 8)/P(x) = 0.01/0.05 = 0.2, thats fairer.

This idea of "How good c is at representing the higher level concept of X" is what we need to make into a tangible expression using the vectors outputs of our neural network.  

One way to tell if two vectors are similar, is to take their dot product. You can also take their cosine similarity. 

A neural network does several linear transformations with nonlinear transformations in between to overall, after training, do some complex transformations. If c_t holds information about z_t+k good enough to predict it, then you should be able to transform c_t into a vector that has a higher dot product with z_t+k than some random z_neg that is not in the future relative to c_t. Thats where this expression comes into play:

$$ e^{z^T_{t+k} W_k c_t} $$


In [1]:

import random
import time
import os
import logging
from timeit import default_timer as timer
import soundfile as sf  

## Libraries
import numpy as np

## Torch
import torch
import torch.nn as nn
from torch.utils import data as datautil
import torch.nn.functional as F
import torch.optim as optim

print('you are using PyTorch version ',torch.__version__)

you are using PyTorch version  1.4.0


# Data

From the website http://www.openslr.org/12/ , the files dev-clean.tar.gz and train-clean-100.tar.gz are downloaded, unpacked and placed into validation and training folders respectively. Each one unpacks into a folder called LibriSpeech but the contents are not the same. Inside this folder there will be a folder called "dev-clean" and "train-clean-100" for the validation and training set respectively. The paths to these folder are what you need to provide to `RawDataset` as in the demonstration below, in order to train our CPC model. 



In [2]:
# Touch the data
# 19 is the speaker ID, 198 is the chapter ID, CHAPTERS.TXT has the name of the book chapter
# LibriSpeech/train-clean-100/19/198/19-198.trans.txt has a line by line text translation of
# all of the flac audio files
path = 'data/training/LibriSpeech/train-clean-100/19/198/19-198-0000.flac'    
audiodata, samplerate = sf.read(path)  # samplerate = 16kHz 
print(samplerate, audiodata, audiodata.shape)

use_cuda = torch.cuda.is_available()
print('use_cuda is', use_cuda)
device = torch.device("cuda" if use_cuda else "cpu")

16000 [0.00436401 0.0032959  0.00314331 ... 0.00466919 0.00604248 0.00598145] (31440,)
use_cuda is True


In [3]:
class RawDataset(datautil.Dataset):
    
    def __init__(self, directory, audio_window):

        self.audio_window = audio_window 
        self.audiolist = []
        self.idx2file = {}
        self.files = []
        # r=root, d=directories, f = files
        for r, d, f in os.walk(directory):
            for file in f:
                if '.flac' in file:
                    self.files.append(os.path.join(r, file))

        for idx, filepath in enumerate(self.files):
            self.idx2file[idx] = filepath

    def __len__(self):
        """Denotes the total number of utterances """
        return len(self.files)

    def __getitem__(self, index):
        filepath = self.idx2file[index]
        audiodata, samplerate = sf.read(filepath)
        utt_len = audiodata.shape[0] 
        # get the index to read part of the utterance into memory 
        index = np.random.randint(utt_len - self.audio_window + 1) 
        return audiodata[index:index+self.audio_window]
    
def save_model(model,name):
    torch.save(model.state_dict(),name)
    
def load_model(model,name):
    model.load_state_dict(torch.load(name))
    

In [4]:
training_set = RawDataset(directory = "data/training/LibriSpeech/train-clean-100", 
                          audio_window = 20480)

validation_set = RawDataset(directory = "data/validation/LibriSpeech/dev-clean", 
                          audio_window = 20480)


In [5]:
class ScheduledOptim(object):
    """A simple wrapper class for learning rate scheduling"""

    def __init__(self, optimizer, n_warmup_steps):
        self.optimizer = optimizer
        self.d_model = 128 
        self.n_warmup_steps = n_warmup_steps
        self.n_current_steps = 0 
        self.delta = 1

    def state_dict(self):
        self.optimizer.state_dict()

    def step(self):
        """Step by the inner optimizer"""
        self.optimizer.step()

    def zero_grad(self):
        """Zero out the gradients by the inner optimizer"""
        self.optimizer.zero_grad()

    def increase_delta(self):
        self.delta *= 2

    def update_learning_rate(self):
        """Learning rate scheduling per step"""

        self.n_current_steps += self.delta
        new_lr = np.power(self.d_model, -0.5) * np.min([
            np.power(self.n_current_steps, -0.5),
            np.power(self.n_warmup_steps, -1.5) * self.n_current_steps])

        for param_group in self.optimizer.param_groups:
            param_group['lr'] = new_lr
        return new_lr


# The Encoder-AutoRegressive Model

The raw waveform is fed to the encoder without preprocessing. The size 20480 audio segment represents a 1.28 second long speech, or audio signal. The 160 downsampling factor for the encoder takes a 20480 sequence and compresses it to sequence length 128. 

When batch_first = True, the GRU input and output shapes are (batch, seq, feature), and this is the shape of z which the GRU takes as input, only that features is equivalent to the channel dimension in 1D CNNs, so z is shpae (batch_size,seq_len,channels). 

The Noise Contrastive Estimation is acheived by using the other samples in the batch as the negative samples. 

What we learn from this implementation is that the expression 

$$W_k c_t$$

in the log-bilinear model, is meant to be an approximation of 

$$z_{t+k}$$

And so the higher the dot product is between the two, the higher is the density ratio estimate f_k, where

$$f_k(x_{t+k}, c_t) = exp(z^T_{t+k} W_k c_t)$$

If you look at the implementation below, you will not see the exponent term, because this exponent term is incorporated into the log softmax term. 

In each training batch, batch_size samples of audio signal is used of size 20480x1. The encoder turns this sequence into shape 128x512 (z). each size 512 vector in z is 0.01 seconds, which is why 128 of them amounts to 1.28 seconds. 

`time_C` is a number chosen between 0 and 128 - K, so if K = 12, then `time_C` is between 0 amd 116. Suppose `time_C` = 100, then the vector c_t of size 256 is an embedding of the first 100 vectors of z or the first 1.0 seconds of the audio signal. 

c_t is used to predict the next K of z_t's in the future. ie z_t+k. If c_t actually holds enough information to predict z_t+k, then some linear transformation W_k should be able to approximate z_t+k from c_t. 

The entire 128 z's are encoded into c's by the same encoder, then a random timstep is chosen from the first 128 - K timsteps in order to predict the next K timesteps, which is why the random timestep `time_C` cannot be higher than 128 - K, or else there will not be a total of K timesteps in the future to compare to. 

# The InfoNCE loss function

If the encoder and the GRU have generated a good representation c_t, then the dot product of W_k c_t with z_t+k should be higher than tha dot product of W_k c_t with zneg_t+k, where zneg_t+k is the z vector by from a different sample in the batch. This is the "negative sample" in "negative sampling". 

In the implementation below the line `zWc = torch.mm(z_t_k[k], torch.transpose(W_c_k[k],0,1))` is performed for each future timestep k in K. 

The z_t_k tensor is the K number of z_t+k's for each sample in the batch.
W_c_k is the K number of vectors that are a result of matrix multiplying Wk with ct that is supposed to approximate z_t_k. The transpose operation is performed on W_c_k in order for the zWc calculation to be a calculation of shape (batch_size, z_size)x(z_size, batch_size) => (batch_size, batch_size). So W_c_k is not just dot producted with the z_t_k it was meant to approximate, it is also dot producted with the z_t_k's fromt he other samples in the batch. The value in position (k,k) of zWc is proportional to how well W_c_k approximates z_t_k, the values in position (k, not k) is proportional to how well W_c_k approximates not the z_t_k from it's own sequence, but the z_t_k from some other sequece, which it should not be good at it it is working properly. 

Suppose batch size is 3 and the first row of zWc are the elements `[0.1,  1,  -0.1]` then the log_softmax is `[-1.4536, -0.5536, -1.6536]`. This means that W_c_k has a weak preference for it's own z_t_k, over the last, 3rd  z_t_k, which is good, but it has a much stronger preference for the 2nd z_t_k than it's own, which is bad. The softmax function forces the entire row to be less and 1 and as a whole sum to 1. If you want the first element in the softmax to be close to 1.0, which you want, since this means that the negative loss will be close to 0, (the negative loss in this case will be +1.4536) then you want not only for the first element to be large, that is not enough, the other elements need to be low. This is why the loss function log_softmax(z_t_k dot W_c_k / sum of z_t_k dot W_c_k for all samples) not only helps to tune your neural network to move in the right direction, but move away from the wrong directions.

Here I show you which line in the code corresponds to the loss function described int he paper:

$$ InfoNCELoss(z_{t = t' \rightarrow t'+k}, c_t) = - E \left[  \log( \frac{f_k(z_{t+k}, c_t)}{\sum_{J} f_k(z_{j}, c_t)} ) \right] $$

$$ = - E \left[  \log( \frac{e^{z^T_{t+k} W_k c_t}}{\sum_{J} e^{z^T_{j} W_k c_t}} ) \right]$$

$$ = - \frac{1}{KJ} \sum_{J}\sum_{K} \log( \frac{e^{z^T_{t+k} W_k c_t}}{\sum_{J} e^{z^T_{j} W_k c_t}} ) $$

$$ = - E \left[  \log( softmax( e^{z^T_{j} W_k c_t)}_i ) \right]  $$

Where i is the index of the correct element in the softmax, the code version of this is:

`nce += torch.sum(torch.diag(logsof_zWc))`

`nce /= -1.*batch_size*self.K` 

The fact that the nce is accumulated over each sample of the batch and each future timestep k, then at the end divided by `batch_size*self.K` represents the expectation. 

The `torch.diag(logsof_zWc)` makes a vector from the diagonal of the `logsof_zWc` matrix, so it is taking only the log softmax values of the correct z_t_k dot W_c_k pairs (this value through the softmax has already incorporated information from the neighboring negative samples). Just like the softmax term is only considering the ratio of the element over the elements including itself:

$$\frac{f_k(z_{t+k}, c_t)}{\sum_{J} f_k(z_{j}, c_t)}$$ 

Here I use z instead of x for simplicity and making the code look like the formula, the paper uses x in the density ratio formula f_k. But z is a direct mapping from x, so hopefully you can see that it is the same. 

In [23]:
class CPC(nn.Module):
    def __init__(self, K, seq_len):
        """
        K: this is K in the paper, the K timesteps in the future relative to
           the last context timestep  are used as targets to teach the model 
           to predict K steps into the future
        """
        super(CPC, self).__init__()

        self.seq_len = seq_len
        self.K = K
        self.c_size = 256
        self.z_size = 512
        self.dwn_fac = 160
        # the downsampling factor of this CNN is 160
        # so the output sequence length is 20480/160 = 128
        self.encoder = nn.Sequential( 
            nn.Conv1d(1, self.z_size, kernel_size=10, stride=5, padding=3, bias=False),
            nn.BatchNorm1d(self.z_size),
            nn.ReLU(inplace=True),
            nn.Conv1d(self.z_size, self.z_size, kernel_size=8, stride=4, padding=2, bias=False),
            nn.BatchNorm1d(self.z_size),
            nn.ReLU(inplace=True),
            nn.Conv1d(self.z_size, self.z_size, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm1d(self.z_size),
            nn.ReLU(inplace=True),
            nn.Conv1d(self.z_size, self.z_size, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm1d(self.z_size),
            nn.ReLU(inplace=True),
            nn.Conv1d(self.z_size, self.z_size, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm1d(self.z_size),
            nn.ReLU(inplace=True)
        )
        
        self.gru = nn.GRU(self.z_size, self.c_size, num_layers = 1, 
                          bidirectional = False, batch_first = True)
        
        # These are all trained
        self.Wk = nn.ModuleList([nn.Linear(self.c_size,self.z_size) for i in range(self.K)])
        self.softmax = nn.Softmax(dim=1)
        self.lsoftmax = nn.LogSoftmax(dim=1)

    def init_hidden(self, batch_size, use_gpu=True):
        if use_gpu: return torch.zeros(1, batch_size, self.c_size).cuda()
        else: return torch.zeros(1, batch_size, self.c_size)

    def forward(self, x, hidden):
        """
        x: torch.float32 shape (batch_size,channels=1,seq_len)
        hidden: torch.float32 
                shape (num_layers*num_directions=1,batch_size,hidden_size=256) 
        """
        batch_size = x.size()[0] 
        
        # input x is shape (batch_size, channels, seq_len), e.g. 8*1*20480
        z = self.encoder(x) 
        # encoded sequence z is shape (batch_size, z_size, seq_len), e.g. 8*512*128
        
        z = z.transpose(1,2) # reshape->(batch_size,seq_len,z_size) for GRU, e.g. 8*128*512
        
        # pick timestep to be the last in the context, time_C, later ones are targets 
        highest = self.seq_len//self.dwn_fac - self.K # 128 -12 = 116
        time_C = torch.randint(highest, size=(1,)).long() # some number between 0 and 116

        # encode_samples (K, batch_size, z_size)ie 12,8,512, 
        z_t_k = z[:, time_C + 1:time_C + self.K + 1, :].clone().cpu().float()
        z_t_k = z_t_k.transpose(1,0)
        
        z_0_T = z[:,:time_C + 1,:] # e.g. size 8*100*512
        output, hidden = self.gru(z_0_T, hidden) # output size e.g. 8*100*256
        c_t = output[:,time_C,:].view(batch_size, self.c_size) # c_t e.g. size 8*256
        
        # For the future K timesteps, predict their z_t+k, 
        W_c_k = torch.empty((self.K, batch_size, self.z_size)).float() # e.g. size 12*8*512
        for k in np.arange(0, self.K):
            linear = self.Wk[k] # c_t is size 256, Wk is a 512x256 matrix 
            W_c_k[k] = linear(c_t) # Wk*c_t e.g. size 8*512
            
        nce = 0 # average over timestep and batch
        for k in np.arange(0, self.K):
            
            # (batch_size, z_size)x(z_size, batch_size) = (batch_size, batch_size)
            zWc = torch.mm(z_t_k[k], torch.transpose(W_c_k[k],0,1)) 
            # print(zWc)
            # total has shape (batch_size, batch_size) e.g. size 8*8
            
            logsof_zWc = self.lsoftmax(zWc)
            #print(logsof_zWc)
            nce += torch.sum(torch.diag(logsof_zWc)) # nce is a tensor
            
        nce /= -1.*batch_size*self.K
        
        argmax = torch.argmax(self.softmax(zWc), dim=0)# softmax not required if argmax 
        correct = torch.sum( torch.eq( argmax, torch.arange(0, batch_size) ) ) 
        accuracy = 1.*correct.item()/batch_size

        return accuracy, nce, hidden

    def predict(self, x, hidden):

        # input sequence is N*C*L, e.g. 8*1*20480
        
        z = self.encoder(x) # encoded sequence is N*C*L, e.g. 8*512*128
        
        z = z.transpose(1,2) # reshape to N*L*C for GRU, e.g. 8*128*512
        output, hidden = self.gru(z, hidden) # output size e.g. 8*128*256

        return output, hidden # return every frame
        #return output[:,-1,:], hidden # only return the last frame per utt

In [24]:
model = CPC(K = 2, seq_len = 20480).to(device)
#load_model(model,"weights/CPC_K2.pth")

In [25]:
params = {}
loader = datautil.DataLoader(training_set, batch_size=3, shuffle=True, **params)

for batch_idx, utterance in enumerate(loader):

    break
    
# needs to be float vs double, add channel dimension
start = timer()
utterance = utterance.float().float().unsqueeze(1).to(device) 
hidden = model.init_hidden(len(utterance), use_gpu=True)
acc, loss, hidden = model(utterance, hidden)
end = timer()
print(end- start) 

0.007570809000753798


# The softmax probability, the density ratio and mutual information
The paper asks you to noitice that the loss function is of the same form as the cross entropy loss 

$$-ylog(P(y|x))$$

where

$$ P(y|x) \approx \frac{f_k(z_{t+k}, c_t)}{\sum_{J} f_k(z_{j}, c_t)}$$  

Returning to the analogy of words and emotions, the loss function is optimized when the model's internal representation of what it means to be angry maximizes P("hate"|angry)/P("hate").

What is the probability that is the angry kind of hate and not one of the other hates like from someone scared or happy? its not P("hate"|angry)/P("hate") cause thats a ratio, not a probability. probabilities need to sum to 1 when you add them together with all the other probabilities, "good", "hate", "the".

The paper presents this as 

$$ P(d=i|X,c_t) = \frac{ \frac{p(word_i|emotion_t)}{p(word_i)} }{ \sum^{N}_{j=1} \frac{p(word_j|emotion_t)}{p(word_j)} } $$

and uses this as a justification for why minimizing the loss function in turns maximizes the density ration, which in turn maximizes the mutual information. The relationship to mutual information is proven in the Appendix. 

In [93]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, data in enumerate(train_loader):
        data = data.float().unsqueeze(1).to(device) # add channel dimension
        optimizer.zero_grad()
        hidden = model.init_hidden(len(data), use_gpu=True)
        acc, loss, hidden = model(data, hidden)

        loss.backward()
        optimizer.step()
        lr = optimizer.update_learning_rate()
        if batch_idx % log_interval == 0:
            print('Train Epoch:{}[{}/{}({:.0f}%)]\tlr:{:.5f}\tAcc:{:.4f}\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), lr, acc, loss.item()))
            
def validation(model, device, data_loader):
    print("Starting Validation")
    model.eval()
    total_loss = 0
    total_acc  = 0 

    with torch.no_grad():
        for data in data_loader:
            data = data.float().unsqueeze(1).to(device) # add channel dimension
            hidden = model.init_hidden(len(data), use_gpu=True)
            acc, loss, hidden = model(data, hidden)
            total_loss += len(data) * loss 
            total_acc  += len(data) * acc

    total_loss /= len(data_loader.dataset) # average loss
    total_acc  /= len(data_loader.dataset) # average acc

    print('===> Validation set: Average loss: {:.4f}\tAccuracy: {:.4f}\n'.format(
                total_loss, total_acc))

    return total_acc, total_loss

In [94]:
model = CPC(K = 2, seq_len = 20480).to(device)
load_model(model,"weights/CPC_K2.pth")

params = {'num_workers': 0,
          'pin_memory': False} if use_cuda else {}

train_loader = datautil.DataLoader(training_set, batch_size = 16, 
                                   shuffle=True, **params) # set shuffle to True

validation_loader = datautil.DataLoader(validation_set, batch_size = 16, 
                                        shuffle=False, **params) # set shuffle to False

In [96]:
optimizer = ScheduledOptim(
    optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()), 
        betas=(0.9, 0.98), eps=1e-09, weight_decay=1e-4, amsgrad=True),
        n_warmup_steps = 50)

## Start training
global_timer = timer() # global timer
log_interval = 200
best_acc = 0
best_loss = np.inf
best_epoch = -1 
epochs = 1

val_acc, val_loss = validation(model, device, validation_loader)

for epoch in range(1, epochs + 1):
    epoch_timer = timer()

    # Train and validate
    train(log_interval, model, device, train_loader, optimizer, epoch)
    val_acc, val_loss = validation(model, device, validation_loader)

    # Save
    if val_acc > best_acc: 
        print("new best val_acc", val_acc)
        save_model(model,"weights/CPC.pth")
        best_acc = max(val_acc, best_acc)
        best_epoch = epoch + 1
    elif epoch - best_epoch > 2:
        optimizer.increase_delta()
        best_epoch = epoch + 1

    end_epoch_timer = timer()
    print("#### End epoch {}/{}, elapsed time: {}".format(epoch,
                                                    epochs, 
                                                   end_epoch_timer - epoch_timer))

## end 
end_global_timer = timer()
print("Total elapsed time: %s" % (end_global_timer - global_timer))

save_model(model,"weights/CPC_K2.pth")

Starting Validation
===> Validation set: Average loss: 3.2256	Accuracy: 0.2279

Train Epoch:1[0/28539(0%)]	lr:0.00025	Acc:0.2500	Loss: 2.091560
Train Epoch:1[3200/28539(11%)]	lr:0.00623	Acc:0.5625	Loss: 1.188438
Train Epoch:1[6400/28539(22%)]	lr:0.00441	Acc:0.3750	Loss: 1.347080
Train Epoch:1[9600/28539(34%)]	lr:0.00361	Acc:0.6875	Loss: 1.061564
Train Epoch:1[12800/28539(45%)]	lr:0.00312	Acc:0.5625	Loss: 0.853313
Train Epoch:1[16000/28539(56%)]	lr:0.00279	Acc:0.8125	Loss: 0.516153
Train Epoch:1[19200/28539(67%)]	lr:0.00255	Acc:0.8750	Loss: 0.399683
Train Epoch:1[22400/28539(78%)]	lr:0.00236	Acc:0.8125	Loss: 0.427373
Train Epoch:1[25600/28539(90%)]	lr:0.00221	Acc:0.9375	Loss: 0.664757
Starting Validation
===> Validation set: Average loss: 0.5609	Accuracy: 0.8320

new best val_acc 0.8320384757676655
#### End epoch 1/1, elapsed time: 119.48127722900063
Total elapsed time: 126.34321080900008


In [22]:
lsoftmax = nn.LogSoftmax(dim=1)
lsoftmax(torch.Tensor([[0.1,  1,  -0.1]]))# [-1.4536, -0.5536, -1.6536]

tensor([[-1.4536, -0.5536, -1.6536]])